# Notebook: Entrenamiento paso a paso del modelo (homework)


- Lectura de datos
- Limpieza y transformaciones
- Separación en entrenamiento/prueba
- Construcción del pipeline (OneHotEncoder + RandomForest)
- Búsqueda de hiperparámetros (GridSearchCV)
- Persistencia del modelo y exportación de métricas


In [21]:
import gzip
import json
import os
import pickle
from typing import Tuple

import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (
    r2_score,
    mean_squared_error,
    median_absolute_error,
    mean_absolute_error,
)
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

In [22]:
RUTA_TRAIN = "../files/input/train_data.csv.zip"
RUTA_TEST = "../files/input/test_data.csv.zip"

RUTA_MODELO = "../files/models/model.pkl.gz"
ARCHIVO_METRICAS = "../files/output/metrics.json"

# Según el enunciado
COLUMNAS_CATEGORICAS = ["Fuel_Type", "Selling_type", "Transmission"]

CV_SPLITS = 10
N_PROCESOS = -1

# Rejilla de hiperparámetros 
REJILLA_PARAMETROS = {
    "selectkbest__k": [3, 4, 5, 6, "all"],
    # Se podría explorar también:
    # "regressor__fit_intercept": [True, False],
}

In [23]:
def limpiar_datos(marco: pd.DataFrame) -> pd.DataFrame:
    """
    Paso 1:
    - Crear Age = 2021 - Year (columna 'Age').
    - Eliminar Year y Car_Name.
    - Eliminar registros con NA (por seguridad).
    """
    marco = marco.copy()

    if "Year" in marco.columns and "Age" not in marco.columns:
        marco["Age"] = 2021 - marco["Year"]

    # Eliminar columnas no requeridas
    for col in ["Year", "Car_Name"]:
        if col in marco.columns:
            marco = marco.drop(columns=[col])

    # Por seguridad, eliminar filas con NA
    marco = marco.dropna()

    return marco

In [24]:
def cargar_datos_limpios(
    ruta_train: str = RUTA_TRAIN,
    ruta_test: str = RUTA_TEST,
):
    import pandas as pd

    train = pd.read_csv(ruta_train, index_col=False, compression="zip")
    test = pd.read_csv(ruta_test, index_col=False, compression="zip")

    train = limpiar_datos(train)
    test = limpiar_datos(test)

    # Present_Price como variable objetivo
    objetivo = "Present_Price"

    x_train = train.drop(columns=[objetivo])
    y_train = train[objetivo].astype(float)

    x_test = test.drop(columns=[objetivo])
    y_test = test[objetivo].astype(float)

    return x_train, y_train, x_test, y_test

In [25]:
def construir_pipeline() -> Pipeline:
    """
    Paso 3:
    Pipeline con:
    - OneHotEncoder para categóricas.
    - MinMaxScaler para numéricas.
    - SelectKBest (f_regression).
    - LinearRegression.
    """
    x_train, _, _, _ = cargar_datos_limpios()

    numeric = [c for c in x_train.columns if c not in COLUMNAS_CATEGORICAS]

    preprocessor = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore"), COLUMNAS_CATEGORICAS),
            ("num", MinMaxScaler(), numeric),
        ],
        remainder="drop",
    )

    regressor = LinearRegression()

    tuberia = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("selectkbest", SelectKBest(score_func=f_regression, k="all")),
            ("regressor", regressor),
        ],
        verbose=False,
    )

    return tuberia

In [26]:
def crear_grid_search(
    tuberia: Pipeline,
    rejilla=None,
    n_folds: int = CV_SPLITS,
    n_jobs: int = N_PROCESOS,
) -> GridSearchCV:
    """
    Paso 4:
    Envuelve el pipeline en un GridSearchCV usando
    'neg_mean_absolute_error' como métrica (MAE).
    """
    if rejilla is None:
        rejilla = REJILLA_PARAMETROS

    buscador = GridSearchCV(
        estimator=tuberia,
        param_grid=rejilla,
        scoring="neg_mean_absolute_error",  # error medio absoluto (con signo negativo)
        cv=n_folds,
        n_jobs=n_jobs,
        verbose=0,
    )

    return buscador

In [27]:
# 5. Guardar / cargar modelo
# ------------------------------------------------------------------------------

def guardar_modelo(modelo) -> None:
    """Guarda el modelo comprimido en RUTA_MODELO."""
    os.makedirs(os.path.dirname(RUTA_MODELO), exist_ok=True)
    with gzip.open(RUTA_MODELO, "wb") as f:
        pickle.dump(modelo, f)

In [28]:
def cargar_modelo():
    """Carga el modelo desde RUTA_MODELO si existe, sino devuelve None."""
    if not os.path.exists(RUTA_MODELO):
        return None
    with gzip.open(RUTA_MODELO, "rb") as f:
        modelo = pickle.load(f)
    return modelo

In [29]:
def ajustar_modelo(busqueda: GridSearchCV) -> None:
    """
    Ajusta el GridSearchCV sobre el conjunto de entrenamiento y guarda
    el mejor modelo.
    """
    x_train, y_train, _, _ = cargar_datos_limpios()

    busqueda.fit(x_train, y_train)

    guardar_modelo(busqueda)

In [30]:
def entrenar_modelo_lineal(rejilla=None) -> None:
    """
    Función de alto nivel para entrenar el modelo de regresión lineal.
    """
    tuberia = construir_pipeline()
    buscador = crear_grid_search(tuberia, rejilla=rejilla)
    ajustar_modelo(buscador)

In [31]:
def evaluar_metricas(modelo, x_train, y_train, x_test, y_test):
    """
    Calcula métricas y retorna dos diccionarios:
    - metrics_train
    - metrics_test

    'mad' se interpreta como median absolute error.
    """
    y_pred_train = modelo.predict(x_train)
    y_pred_test = modelo.predict(x_test)

    def dic_metricas(y_true, y_pred, dataset):
        return {
            "type": "metrics",
            "dataset": dataset,
            "r2": float(r2_score(y_true, y_pred)),
            "mse": float(mean_squared_error(y_true, y_pred)),
            # mad = median absolute error (no mean)
            "mad": float(median_absolute_error(y_true, y_pred)),
        }

    metrics_train = dic_metricas(y_train, y_pred_train, "train")
    metrics_test = dic_metricas(y_test, y_pred_test, "test")

    return metrics_train, metrics_test

In [32]:
def guardar_reporte(
    metrics_train,
    metrics_test,
) -> None:
    """
    Paso 6:
    Escribe ARCHIVO_METRICAS con 2 líneas (train y test).
    """
    os.makedirs(os.path.dirname(ARCHIVO_METRICAS), exist_ok=True)
    with open(ARCHIVO_METRICAS, "w", encoding="utf-8", newline="\n") as f:
        f.write(json.dumps(metrics_train) + "\n")
        f.write(json.dumps(metrics_test) + "\n")

In [33]:
def imprimir_reporte(metrics_train, metrics_test) -> None:
    """Imprime un resumen compacto de métricas (test (train))."""

    def fmt(nombre, val_test, val_train):
        return f"{nombre:>10}: {val_test:.4f} ({val_train:.4f})"

    print("-" * 60)
    print("Metrics summary (test (train))")
    print("-" * 60)
    print(fmt("R2", metrics_test["r2"], metrics_train["r2"]))
    print(fmt("MSE", metrics_test["mse"], metrics_train["mse"]))
    print(fmt("MAD", metrics_test["mad"], metrics_train["mad"]))
    print("-" * 60)

In [35]:
def verificar_estimador() -> None:
    """
    Carga datos, evalúa el modelo guardado y genera ARCHIVO_METRICAS.
    Útil para revisar que todo quedó bien.
    """
    x_train, y_train, x_test, y_test = cargar_datos_limpios()
    modelo = cargar_modelo()
    if modelo is None:
        raise RuntimeError("No se encontró el modelo entrenado en files/models.")

    metrics_train, metrics_test = evaluar_metricas(
        modelo, x_train, y_train, x_test, y_test
    )

    guardar_reporte(metrics_train, metrics_test)
    imprimir_reporte(metrics_train, metrics_test)

In [37]:
def imprimir_parametros() -> None:
    """Imprime todos los parámetros del GridSearchCV guardado."""
    modelo = cargar_modelo()
    if modelo is None:
        print("No model found.")
        return
    print("Get model parameters:")
    for param, value in modelo.get_params().items():
        print(f"  {param}: {value}")

In [38]:
def imprimir_mejores_parametros() -> None:
    """Imprime los mejores hiperparámetros encontrados por GridSearchCV."""
    modelo = cargar_modelo()
    if modelo is None:
        print("No model found.")
        return
    if not hasattr(modelo, "best_params_"):
        print("El modelo cargado no tiene atributo best_params_.")
        return
    print("Best model parameters:")
    for param, value in modelo.best_params_.items():
        print(f"  {param}: {value}")

In [39]:
# ------------------------------------------------------------------------------

if __name__ == "__main__":
    # Entrena el modelo y lo guarda en files/models/model.pkl.gz
    entrenar_modelo_lineal()

    # Evalúa el modelo guardado y genera files/output/metrics.json
    verificar_estimador()

    # imprime los mejores hiperparámetros encontrados
    imprimir_mejores_parametros()

------------------------------------------------------------
Metrics summary (test (train))
------------------------------------------------------------
        R2: 0.7326 (0.8917)
       MSE: 32.5667 (5.8746)
       MAD: 1.5034 (1.0929)
------------------------------------------------------------
Best model parameters:
  selectkbest__k: all
